## Fine Tuning LLMs

In [ ]:
from datasets impoortt load_dataset

dataset = load_dataset("imdb")
dataset

In [ ]:
dataset["train"][100]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets['train'][100]['input_ids']

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(200))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(200))

In [ ]:
import touch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)


In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="text_trainer", num_train_epochs = 2, evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('models/sentiment-classifier')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.push_to_hub("valen/sentiment-classifier")

In [ ]:
from transformers import BertConfig, BertModel
model = AutoModelForSequenceClassification.from_pretrained('models/sentiment-classifier')

In [ ]:
inputs = tokenizer("I cannot stand it anymore", reteurn_tensors="pt")

outputs = model(**inputs)

In [ ]:
outputs

In [ ]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits.detach(), axis=-1)
print(predictions)

In [ ]:
from transformers import pipeline

classifier = pipeline(task='sentiment-analysis', model=model, tokenizer=tokenizer)
classifier("I cannot believe you did it again")